In [5]:
# Nov. 11, 2021, Xiaole Zhang
# Download meteo data from ecmwf 
# Dec, 29, 2021, Xiaole Zhang
# fix the date bugs when combine the files
import cdsapi
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import os
from pathlib import Path
import calendar 

# covert number to formated string
def number2String(f):
    return '%02d' % f

c = cdsapi.Client() 

# [up, left, down, right]
regionName = 'Xian'
regionBound = {'Beijing': [50, 105, 30, 127],\
               'Xian': [44.25, 97.5, 24.25, 119.5]}

# data for Beijing
initialYear = 2020
initialMonth = 12
numberOfMonths = 2
initialDate = date(initialYear, initialMonth, 1) 

#targetYear = 2021
#targetMonths = [1,2]
#monthsList = list(map(number2String, targetMonths))

targetDays = range(1,32)
daysList = list(map(number2String, targetDays))

# to download the meteo day by month to meet the volume requirement
for currentMonthNum in range(numberOfMonths):
    currentDay = initialDate+ relativedelta(months=currentMonthNum)#date(targetYear, currentMonth, 1) 
    initalDay = currentDay 
    
    year = '%4d' % currentDay.year
    month = '%02d' % currentDay.month
    day = '%02d' % currentDay.day
    print('Current Day:' + currentDay.isoformat())
    
    # download single level data
    fileName = regionName + '/'+'SingleLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    filePath = Path(regionName + '/')
    filePath.mkdir(parents=True, exist_ok=True)
    c.retrieve( 
        'reanalysis-era5-single-levels', 
        { 
            'product_type': 'reanalysis', 
            'format': 'grib', 
            'variable': [ 
                '2m_temperature', 'boundary_layer_height', 'eastward_turbulent_surface_stress', 
                'evaporation', 'high_cloud_cover', 'medium_cloud_cover', 
                'northward_turbulent_surface_stress', 'skin_temperature', 'surface_sensible_heat_flux', 
                'surface_solar_radiation_downwards', 'surface_pressure', 'convective_precipitation','large_scale_precipitation','volumetric_soil_water_layer_1', 
            ], 
            'year': year, 
            'month': month,
            'day': daysList, 
            'time': [ 
                '00:00', '01:00', '02:00', 
                '03:00', '04:00', '05:00', 
                '06:00', '07:00', '08:00', 
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ], 
            'area': regionBound[regionName], 
        }, 
        fileName) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file'
    os.system("grib_filter "+ruleFile+" "+fileName)
    
    ###############################################
    # download 3d data
    fileName3d =  regionName + '/'+'PressureLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    c.retrieve( 
        'reanalysis-era5-pressure-levels', 
        { 
        'product_type': 'reanalysis', 
        'variable': [ 
        'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity', 
        'temperature', 'u_component_of_wind', 'v_component_of_wind', 
        ], 
        'pressure_level': [ 
        '500', '550', '600', 
        '650', '700', '750', 
        '775', '800', '825', 
        '850', '875', '900', 
        '925', '950', '975', 
        '1000', 
        ], 
        'year': year, 
        'month': month,
        'day': daysList, 
        'time': [ 
        '00:00', '01:00', '02:00', 
        '03:00', '04:00', '05:00', 
        '06:00', '07:00', '08:00', 
        '09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00',
        '15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00',
        '21:00', '22:00', '23:00', 
        ], 
        'area': regionBound[regionName], 
        'format': 'grib', 
        }, 
        fileName3d) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file_pressure'
    os.system("grib_filter "+ruleFile+" "+fileName3d)
    
    ###########################
    # combine the daily 2d and 3d data files into single files and delete the splitted data
    # need to install the grib api tools
    # sudo apt-get install libgrib-api-tools

    dayRange = calendar.monthrange(currentDay.year, currentDay.month)
    for day in range(0, dayRange[1]):
        currentDay = initalDay + timedelta(days=day) 
        print('Current Day:' + currentDay.isoformat())
        fileName3d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'_pressureLevels.grib'
        fileName2d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'.grib'
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        os.system("cat "+fileName2d+" "+fileName3d+" > "+fileNameAll)
        os.system("rm "+fileName2d+" "+fileName3d) 
        
    if(currentMonthNum==0):
        currentDay = initalDay
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        currentDay = initalDay + timedelta(days=-1)
        fileNameAllTmp = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        fileName2d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'.grib'
        os.system("cat "+fileNameAll+" > "+fileNameAllTmp)
        os.system("rm "+fileName2d) 
    else:       
        currentDay = initalDay + timedelta(days=-1)
        fileName2d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'.grib'
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        # ">>" should be used instead of ">", otherwise the input file will be erased
        os.system("cat "+fileNameAll+" "+fileName2d+" >> "+fileNameAll)
        os.system("rm "+fileName2d)
   #fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
   #os.system("cat "+fileName+" "+fileName3d+" > "+fileNameAll)
   #os.system("rm "+fileName+" "+fileName3d)

2021-12-28 22:49:40,980 INFO Welcome to the CDS
2021-12-28 22:49:40,980 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Current Day:2020-12-01


2021-12-28 22:49:41,086 INFO Request is queued
2021-12-28 22:49:42,118 INFO Request is running
2021-12-28 22:53:59,259 INFO Request is completed
2021-12-28 22:53:59,260 INFO Downloading https://download-0010.copernicus-climate.eu/cache-compute-0010/cache/data7/adaptor.mars.internal-1640728181.2570467-11068-3-c3e09d2c-bc86-49ed-8392-08f585529c35.grib to Xian/SingleLevel_20201201.grib (145.4M)
2021-12-28 22:54:11,694 INFO Download rate 11.7M/s 
2021-12-28 22:54:11,795 INFO Welcome to the CDS
2021-12-28 22:54:11,796 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-12-28 22:54:11,832 INFO Request is queued
2021-12-28 22:54:12,864 INFO Request is running
2021-12-28 23:14:31,727 INFO Request is completed
2021-12-28 23:14:31,728 INFO Downloading https://download-0008.copernicus-climate.eu/cache-compute-0008/cache/data6/adaptor.mars.internal-1640728451.8905907-17147-3-d3930993-bafc-4349-8aed-cb486da935f9.grib to Xian/PressureLevel_

Current Day:2020-12-01
Current Day:2020-12-02
Current Day:2020-12-03
Current Day:2020-12-04
Current Day:2020-12-05
Current Day:2020-12-06
Current Day:2020-12-07
Current Day:2020-12-08
Current Day:2020-12-09
Current Day:2020-12-10
Current Day:2020-12-11
Current Day:2020-12-12
Current Day:2020-12-13
Current Day:2020-12-14
Current Day:2020-12-15
Current Day:2020-12-16
Current Day:2020-12-17
Current Day:2020-12-18
Current Day:2020-12-19
Current Day:2020-12-20
Current Day:2020-12-21
Current Day:2020-12-22
Current Day:2020-12-23
Current Day:2020-12-24
Current Day:2020-12-25
Current Day:2020-12-26
Current Day:2020-12-27


2021-12-28 23:16:01,830 INFO Welcome to the CDS
2021-12-28 23:16:01,831 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-12-28 23:16:01,868 INFO Request is queued


Current Day:2020-12-28
Current Day:2020-12-29
Current Day:2020-12-30
Current Day:2020-12-31
Current Day:2021-01-01


2021-12-28 23:16:02,900 INFO Request is running
2021-12-28 23:20:20,042 INFO Request is completed
2021-12-28 23:20:20,042 INFO Downloading https://download-0002.copernicus-climate.eu/cache-compute-0002/cache/data8/adaptor.mars.internal-1640729761.9408574-29810-7-6f97df3b-37dd-4e89-9a0c-f39176d29c93.grib to Xian/SingleLevel_20210101.grib (145.4M)
2021-12-28 23:20:27,171 INFO Download rate 20.4M/s 
2021-12-28 23:20:27,246 INFO Welcome to the CDS
2021-12-28 23:20:27,247 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-12-28 23:20:27,283 INFO Request is queued
2021-12-28 23:20:28,315 INFO Request is running
2021-12-28 23:46:47,794 INFO Request is completed
2021-12-28 23:46:47,794 INFO Downloading https://download-0015.copernicus-climate.eu/cache-compute-0015/cache/data7/adaptor.mars.internal-1640730027.3399293-16512-20-1f9a929f-8ebd-4152-aeb3-b2c50efb95d9.grib to Xian/PressureLevel_20210101.grib (915.5M)
2021-12-28 23:47:42,235

Current Day:2021-01-01
Current Day:2021-01-02
Current Day:2021-01-03
Current Day:2021-01-04
Current Day:2021-01-05
Current Day:2021-01-06
Current Day:2021-01-07
Current Day:2021-01-08
Current Day:2021-01-09
Current Day:2021-01-10
Current Day:2021-01-11
Current Day:2021-01-12
Current Day:2021-01-13
Current Day:2021-01-14
Current Day:2021-01-15
Current Day:2021-01-16
Current Day:2021-01-17
Current Day:2021-01-18
Current Day:2021-01-19
Current Day:2021-01-20
Current Day:2021-01-21
Current Day:2021-01-22
Current Day:2021-01-23
Current Day:2021-01-24
Current Day:2021-01-25
Current Day:2021-01-26
Current Day:2021-01-27
Current Day:2021-01-28
Current Day:2021-01-29
Current Day:2021-01-30
Current Day:2021-01-31
